**disallowedSynonym.ipynb**

narrowSynonym or broadSynonym defined in Format subontology.

**Documentation:** https://github.com/edamontology/edamverify/blob/master/docs/disallowedSynonym.md

**NB.1: - Running the notebook**
The directory containing the ``EDAM_dev.owl`` file must be defined by ``EDAM_PATH`` environment variable.

The script requires the test to be run from a subdirectory of ``EDAM_PATH`` (hence ``'../EDAM_dev.owl'`` below)

In [14]:
import os
from rdflib import ConjunctiveGraph, Namespace
import json

# Constants for script error reporting as per https://github.com/edamontology/edamverify.
NOERR = "NOERR"
INFO  = "INFO"
WARN  = "WARN"
ERROR = "ERROR"

#Load EDAM_dev.owl from GitHub into an RDF graph.
print("Loading graph ...", end="")
g = ConjunctiveGraph()
g.load(os.environ.get('EDAM_PATH', '../EDAM_dev.owl'), format='xml')
# g.load('https://raw.githubusercontent.com/edamontology/edamontology/master/EDAM_dev.owl', format='xml')
# g.load('EDAM_dev.owl')
g.bind('edam', Namespace('http://edamontology.org#'))
print("done!")

Loading graph ...done!


Define SPARQL query to extract ID, term, and (if available) narrow and broad synonyms of all Format concepts. Run the query.

In [15]:
# Compile SPARQL query
query_term = """
SELECT ?id ?term ?narrow_syn ?broad_syn WHERE
{
?id rdfs:label ?term .
OPTIONAL {?id oboInOwl:hasNarrowSynonym ?narrow_syn .}
OPTIONAL {?id oboInOwl:hasBroadSynonym ?broad_syn .}
FILTER regex(str(?id), "format_")
}
"""

# Run SPARQL query and collate results
errfound = False    
report = list()
results = g.query(query_term)

Analyse results of query.

**NB:**
**CAUTION:** If a (part of a) SPARQL query cannot be satisfied then the associated SPARQL variable will get assigned the value of "None" - *note* "None" as a *string value*.  This should not be confused with the **None** (null) Python keyword. 



In [16]:
for r in results :
    
    id     = str(r['id'])
    term   = str(r['term'])
    narrow_syn = str(r['narrow_syn']) 
    broad_syn = str(r['broad_syn']) 
            
#    print(str(r['id']), "   ", id, "   narrow:", narrow_syn, "   broad:", broad_syn)
   
    # "None" string is return from SPARQL query where value was not found
    if narrow_syn != "None":
        errfound = True
        report.append("Disallowed narrowSynonym ::: " + id +  ' (' + term + ')' + " ::: " + "narrowSynonym:" + "   " + narrow_syn)

    if broad_syn != "None":
        errfound = True
        report.append("Disallowed broadSynonym ::: " + id +  ' (' + term + ')' + " ::: " + "broadSynonym:" + "   " + broad_syn)

Write report and return appropriate value.

In [17]:
report_obj = {}
report_obj['test_name'] = 'disallowedSynonym'
report_obj['comment'] = 'Synonym (disallowed) defined for one or more Format concepts.'

if errfound:
    report_obj['status'] = ERROR
    report_obj['reason'] = report
else:
    report_obj['status'] = NOERR

report_json = json.dumps(report_obj, indent=4)
print(report_json)

{
    "test_name": "disallowedSynonym",
    "comment": "Synonym (disallowed) defined for one or more Format concepts.",
    "status": "NOERR"
}
